# Projet Diagnostic

## Chargement et brève analyse descriptive

Importation des librairies nécessaires

In [1]:
import pandas as pd
import os
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb 
import sys
import numpy as np

Chargement des données :

In [2]:
ot_odr_filename = os.path.join(".", "OT_ODR.csv.bz2")
ot_odr_df = pd.read_csv(ot_odr_filename,
                        compression="bz2",
                        sep=";")

equipements_filename = os.path.join(".", 'EQUIPEMENTS.csv')
equipements_df = pd.read_csv(equipements_filename,
                             sep=";")

Analyse des modalités des variables de signalement :

In [3]:
var_sig = ["SIG_ORGANE", "SIG_CONTEXTE", "SIG_OBS"]
ot_odr_df[var_sig].describe()

,SIG_ORGANE,SIG_CONTEXTE,SIG_OBS
count,506558,506558,506558
unique,116,5509,59
top,ECLAIRAGE FEUX EXTERIEURS,ARRIERE,NE FONCTIONNE PAS
freq,189559,32095,150740


Analyse des modalités des variables type travail et OdR :

In [4]:
var_odr = ["TYPE_TRAVAIL", "ODR_LIBELLE"]
ot_odr_df[var_odr].describe()

,TYPE_TRAVAIL,ODR_LIBELLE
count,506558,506558
unique,12,1410
top,ELECTRICITE,REMPLACEMENT D'UNE LAMPE DE CODE / PHARE
freq,258129,40302


In [5]:
ot_odr_df["TYPE_TRAVAIL"].unique()

array(['CARROSSERIE', 'ELECTRICITE', 'MECANIQUE', 'MISE AU POINT',
       'EQUIPEMENT EMBARQUE', 'EXTINCTEUR', 'STATION SERVICE', 'ROUE',
       'CLIMATISATION', 'BATTERIE', 'DIVERS', 'CONTROLE'], dtype=object)

## Préparation des données

On change le type des variables catégorielles avec la méthode `.astype('category')` :

In [6]:
var_cat = ['ODR_LIBELLE', 'TYPE_TRAVAIL',
           'SYSTEM_N1', 'SYSTEM_N2', 'SYSTEM_N3', 
           'SIG_ORGANE', 'SIG_CONTEXTE', 'SIG_OBS', 'LIGNE']
for var in var_cat:
    ot_odr_df[var] = ot_odr_df[var].astype('category')

ot_odr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506558 entries, 0 to 506557
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   OT_ID          506558 non-null  object  
 1   ODR_ID         506558 non-null  object  
 2   ODR_LIBELLE    506558 non-null  category
 3   TYPE_TRAVAIL   506558 non-null  category
 4   DUREE_TRAVAIL  506558 non-null  float64 
 5   SYSTEM_N1      506558 non-null  category
 6   SYSTEM_N2      506558 non-null  category
 7   SYSTEM_N3      506558 non-null  category
 8   EQU_ID         506558 non-null  object  
 9   DATE_OT        506558 non-null  object  
 10  KILOMETRAGE    506557 non-null  float64 
 11  SIG_ORGANE     506558 non-null  category
 12  SIG_CONTEXTE   506558 non-null  category
 13  SIG_OBS        506558 non-null  category
 14  LIGNE          506558 non-null  category
dtypes: category(9), float64(2), object(4)
memory usage: 29.7+ MB


In [7]:
var_cat = ['MODELE', 'CONSTRUCTEUR', 'MOTEUR']
for var in var_cat:
    equipements_df[var] = equipements_df[var].astype('category')

equipements_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4887 entries, 0 to 4886
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   EQU_ID        4887 non-null   object  
 1   MODELE        4887 non-null   category
 2   CONSTRUCTEUR  4887 non-null   category
 3   MOTEUR        4887 non-null   category
dtypes: category(3), object(1)
memory usage: 58.4+ KB


In [8]:
joined_df = ot_odr_df.set_index('EQU_ID').join(equipements_df.set_index('EQU_ID'))

In [9]:
joined_df = joined_df.reset_index(drop="True")
joined_df

,OT_ID,ODR_ID,ODR_LIBELLE,TYPE_TRAVAIL,DUREE_TRAVAIL,SYSTEM_N1,SYSTEM_N2,SYSTEM_N3,DATE_OT,KILOMETRAGE,SIG_ORGANE,SIG_CONTEXTE,SIG_OBS,LIGNE,MODELE,CONSTRUCTEUR,MOTEUR
0,OT000002306,OM000002463,REMPLACEMENT D'UN REGLEUR DE DEBIT DE PORTE,ELECTRICITE,0.50,EQUIPEMENT DE CARROSSERIE,PORTE,COMMANDE PORTE,2012-06-29 22:41:19,324206.608000,PORTE,PLATE FORME,FONCTIONNE MAL,L0385,MD037,C007,MT010
1,OT000005633,OM000006163,REMPLACEMENT DE L'ELECTROVANNE DE DEGIVRAGE,MECANIQUE,1.00,EQUIPEMENT CLIMATIQUE,CHAUFFAGE/DEGIVRAGE,ORGANE DE CHAUFFAGE/DEGIVRAGE,2013-03-05 18:22:19,349306.448000,DIRECTION,EN VIRAGE/A L'ARRET,A-COUPS,L0385,MD037,C007,MT010
2,OT000006231,OM000006857,REMPLACEMENT D'UN HABILLAGE INT DE BAIE,CARROSSERIE,0.25,EQUIPEMENT DE CARROSSERIE,AMENAGEMENT INT,HABILLAGE INT,2013-04-05 20:17:29,352589.208000,GLACE/BAIE,DROIT/ARRIERE,TAGUE,L0385,MD037,C007,MT010
3,OT000012829,OM000014475,REMPLACEMENT D'UN PORTILLON LAT,CARROSSERIE,0.41,EQUIPEMENT DE CARROSSERIE,ELEMENT CARROSSERIE EXT,CARROSSERIE LAT,2013-11-07 19:10:57,372248.368000,PANNEAU,GAUCHE/LATERAL/BAS,CASSE,L0385,MD037,C007,MT010
4,OT000013786,OM000015291,REMPLACEMENT D'UN ECLAIRAGE INT,ELECTRICITE,0.06,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION INT,2013-11-21 19:15:46,374007.544000,DEMANDE ARRET,BAS/TABLEAU DE BORD/DROIT,VOYANT HS,L0385,MD037,C007,MT010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506553,OT000390943,OM000486456,REMPLACEMENT DU CONCENTRATEUR PEAGE,EQUIPEMENT EMBARQUE,0.08,EQUIPEMENT EMBARQUE,TELEBILLETIQUE,CONCENTRATEUR DE PEAGE,2019-07-02 21:15:31,923.702323,AVTT,AVANT/DROIT,FONCTIONNE MAL,L0077,MD032,C006,MT016
506554,OT000400206,OM000498614,REMPLACEMENT DE LA RADIO,EQUIPEMENT EMBARQUE,0.05,EQUIPEMENT EMBARQUE,AIDE A L'EXPLOITATION-MAINTENANCE,RADIO,2019-08-29 20:53:23,5418.005056,RADIO,POSTE CONDUITE,NE FONCTIONNE PAS,L0077,MD032,C006,MT016
506555,OT000400557,OM000499548,REMPLACEMENT D'UN FEU DE GABARIT,ELECTRICITE,0.04,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,2019-09-01 06:07:41,4337.170024,ECLAIRAGE FEUX EXTERIEURS,BAS/GAUCHE/LATERAL,CASSE,L0509,MD064,C007,MT024
506556,OT000405447,OM000505789,REMPLACEMENT D'UN PASSE SANS CONTACT,EQUIPEMENT EMBARQUE,0.06,EQUIPEMENT EMBARQUE,TELEBILLETIQUE,PASSE SANS CONTACT,2019-09-26 19:52:05,2599.051019,AVTT,EN MONTEE,VOYANT HS,L0178,MD021,C011,MT051


In [10]:
joined_df_filtered = joined_df.select_dtypes(exclude=['object'])
joined_df_filtered

,ODR_LIBELLE,TYPE_TRAVAIL,DUREE_TRAVAIL,SYSTEM_N1,SYSTEM_N2,SYSTEM_N3,KILOMETRAGE,SIG_ORGANE,SIG_CONTEXTE,SIG_OBS,LIGNE,MODELE,CONSTRUCTEUR,MOTEUR
0,REMPLACEMENT D'UN REGLEUR DE DEBIT DE PORTE,ELECTRICITE,0.50,EQUIPEMENT DE CARROSSERIE,PORTE,COMMANDE PORTE,324206.608000,PORTE,PLATE FORME,FONCTIONNE MAL,L0385,MD037,C007,MT010
1,REMPLACEMENT DE L'ELECTROVANNE DE DEGIVRAGE,MECANIQUE,1.00,EQUIPEMENT CLIMATIQUE,CHAUFFAGE/DEGIVRAGE,ORGANE DE CHAUFFAGE/DEGIVRAGE,349306.448000,DIRECTION,EN VIRAGE/A L'ARRET,A-COUPS,L0385,MD037,C007,MT010
2,REMPLACEMENT D'UN HABILLAGE INT DE BAIE,CARROSSERIE,0.25,EQUIPEMENT DE CARROSSERIE,AMENAGEMENT INT,HABILLAGE INT,352589.208000,GLACE/BAIE,DROIT/ARRIERE,TAGUE,L0385,MD037,C007,MT010
3,REMPLACEMENT D'UN PORTILLON LAT,CARROSSERIE,0.41,EQUIPEMENT DE CARROSSERIE,ELEMENT CARROSSERIE EXT,CARROSSERIE LAT,372248.368000,PANNEAU,GAUCHE/LATERAL/BAS,CASSE,L0385,MD037,C007,MT010
4,REMPLACEMENT D'UN ECLAIRAGE INT,ELECTRICITE,0.06,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION INT,374007.544000,DEMANDE ARRET,BAS/TABLEAU DE BORD/DROIT,VOYANT HS,L0385,MD037,C007,MT010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506553,REMPLACEMENT DU CONCENTRATEUR PEAGE,EQUIPEMENT EMBARQUE,0.08,EQUIPEMENT EMBARQUE,TELEBILLETIQUE,CONCENTRATEUR DE PEAGE,923.702323,AVTT,AVANT/DROIT,FONCTIONNE MAL,L0077,MD032,C006,MT016
506554,REMPLACEMENT DE LA RADIO,EQUIPEMENT EMBARQUE,0.05,EQUIPEMENT EMBARQUE,AIDE A L'EXPLOITATION-MAINTENANCE,RADIO,5418.005056,RADIO,POSTE CONDUITE,NE FONCTIONNE PAS,L0077,MD032,C006,MT016
506555,REMPLACEMENT D'UN FEU DE GABARIT,ELECTRICITE,0.04,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,4337.170024,ECLAIRAGE FEUX EXTERIEURS,BAS/GAUCHE/LATERAL,CASSE,L0509,MD064,C007,MT024
506556,REMPLACEMENT D'UN PASSE SANS CONTACT,EQUIPEMENT EMBARQUE,0.06,EQUIPEMENT EMBARQUE,TELEBILLETIQUE,PASSE SANS CONTACT,2599.051019,AVTT,EN MONTEE,VOYANT HS,L0178,MD021,C011,MT051


In [18]:
joined_df_filtered = joined_df_filtered[joined_df_filtered['KILOMETRAGE'].notna()]
# Définir des intervalles personnalisés
bins = [0, 50000, 100000, 150000, 200000, 250000, 300000, float("inf")]
labels = ['C:0-50k', 'C:50k-100k', 'C:100k-150k', 'C:150k-200k', 'C:200k-250k', 'C:250k-300k', 'C:300k+']
#Genie n'aime pas quand une catégorie commence par un chiffre


# Définir une fonction pour attribuer des catégories
def categorize_kilometrage(km):
    if km < 50000:
        return 'C:0-50k'
    elif km < 100000:
        return 'C:50k-100k'
    elif km < 150000:
        return 'C:100k-150k'
    elif km < 200000:
        return 'C:150k-200k'
    elif km < 250000:
        return 'C:200k-250k'
    elif km < 300000:
        return 'C:250k-300k'
    else:
        return 'C:300k+'

# Appliquer la fonction pour créer la nouvelle colonne
joined_df_filtered['KM_GROUP'] = joined_df_filtered['KILOMETRAGE'].apply(categorize_kilometrage).astype('category')

In [19]:
joined_df_filtered['KM_GROUP']

0         C:300k+
1         C:300k+
2         C:300k+
3         C:300k+
4         C:300k+
           ...   
506553    C:0-50k
506554    C:0-50k
506555    C:0-50k
506556    C:0-50k
506557    C:0-50k
Name: KM_GROUP, Length: 506557, dtype: category
Categories (7, object): ['C:0-50k', 'C:100k-150k', 'C:150k-200k', 'C:200k-250k', 'C:250k-300k', 'C:300k+', 'C:50k-100k']

## Création d'un premier modèle

Nous proposons de créer un modèle permettant d'expliquer la variable SYSTEM_N1 à partir de SIG_OBS.

On commence par créer les variables du réseau.

In [20]:
var_to_model = ["SYSTEM_N1", "SIG_OBS", "SIG_ORGANE", "KM_GROUP"]

var_bn = {}
for var in var_to_model:
    nb_values = len(joined_df_filtered[var].cat.categories)
    var_bn[var] = gum.LabelizedVariable(var, var, nb_values)
    

On renseigne les modalités des variables pour être fidèle à celles des données.

In [21]:
for var in var_bn:
    for i, modalite in enumerate(joined_df_filtered[var].cat.categories):
        var_bn[var].changeLabel(i, modalite)
    print(var_bn[var])

SYSTEM_N1:Labelized({DIVERS|EQUIPEMENT CHASSIS|EQUIPEMENT CLIMATIQUE|EQUIPEMENT DE CARROSSERIE|EQUIPEMENT DE FREINAGE|EQUIPEMENT DE MOTORISATION|EQUIPEMENT DE TRANSMISSION|EQUIPEMENT ELECTRIQUE|EQUIPEMENT EMBARQUE|EQUIPEMENT PNEUMATIQUE})
SIG_OBS:Labelized({A-COUPS|ABSENT|ALLUME|ARRACHE|ASPIRER|BALANCE|BLESSE|BLOQUE|BROUTE|BRUTAL|BRUYANT|CALE|CASSE|CLAQUE|CLIGNOTE|CREVE|DEBOITE|DECLENCHEMENT|DEGONFLE|DELOCALISE|DEREGLE|DESSERRE|DUR|ENFONCE|FAUX CONTACT|FENDU|FERME MAL|FONCTIONNE MAL|FUITE|FUME/CHAUFFE|INCOMPLET|INSUFFISANT|INTERMITTENT|INTERRUPTEUR H.S|JEU|LACERE|MANQUE|MAUVAIS|N'EMET PAS|N'ENCRE PAS|NE FONCTIONNE PAS|NE TIRE PAS|ODEUR|OUVRE MAL|PATINE|RAYE|REFUSE TITRE|SALE|SECOUSSE|TAGUE|TANGUE|TEMOIN OUVERTURE H.S|TIRE A DROITE|TIRE A GAUCHE|TORDU|USE/VIDE|VIBRE|VOYANT ALLUME|VOYANT HS})
SIG_ORGANE:Labelized({ABS/ASR|ACCELERATEUR ELECTRIQUE|ACCOUDOIR|AERATEURS DE TOITURE|AIR|ALARME DISCRETE|ANNONCEUR SONORE|ANNONCEUR VISUEL|ARRET MOTEUR|ARRET URGENCE|ARTICULATION REMORQUE|AUTO-RADIO

Ensuite, on crée le réseau bayésien et on ajoute les variables sélectionnées.

In [22]:
bn = gum.BayesNet("modèle simple")

for var in var_bn.values():
    bn.add(var)

Pour finir, on fixe la structure.

In [23]:
bn.addArc("SYSTEM_N1", "SIG_OBS")
bn.addArc("SYSTEM_N1", "SIG_ORGANE")
bn.addArc("SYSTEM_N1", "KM_GROUP")

Nous pouvons utiliser temporairement un notebook Jupyter afin de vérifier graphiquement la structure et les LPC de notre modèle.

* Pas sur de ce qu'il faut faire ici. Y'a l'import de gnb a utiliser peut-être ? *

In [24]:
bn

(pyAgrum.BayesNet<double>@000002553F78FED0) BN{nodes: 4, arcs: 3, domainSize: 479080, dim: 1799, mem: 14Ko 304o}

Loi de probabilité de SIG_OBS, SIG_ORGANE, KM_GROUP :

In [25]:
bn.cpt("SIG_OBS")

(pyAgrum.Potential<double>@000002551F43ABB0) 
      ||  SIG_OBS                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
SYSTEM||A-COUPS  |ABSENT   |ALLUME   |ARRACHE  |ASPIRER  |BALANCE  |BLESSE   |BLOQUE   |BROUTE   |BRUTAL   |BRUYANT  |CALE     |CASSE    |CLAQUE   |CLIGNOTE |CREVE    |DEBOITE  |DECLENCHE|DEGONFLE |DELOCALIS|DEREGLE  |DESSERRE |DUR      |ENFONCE  |FAUX CONT|FENDU    |FERME MAL|FONCTIONN|FUITE    |FUME/CHAU|INCOMPLET|INSUFFISA|INTERMITT|INTERRUPT|JEU      |LACERE   |MANQUE   |MAUVAIS  |N'EMET PA|N'ENCRE P|NE FONCTI|NE TIRE P|ODEUR    |OUVRE MAL|PATINE   |RAYE     |REFUSE TI|SALE     |SECOUSSE |TAGUE    |TANGUE   |TEMOIN OU|TIRE A DR|TIRE A GA|TORDU    |USE/VIDE |VIBRE    |VOYANT AL|VOYANT HS|
------||---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|
DIVERS|| 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  |
EQUIPE|| 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  |
EQUIPE|| 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  |
EQUIPE|| 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0

In [26]:
bn.cpt("SIG_ORGANE")

(pyAgrum.Potential<double>@000002551F43AD90) 
      ||  SIG_ORGANE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
SYSTEM||ABS/ASR  |ACCELERAT|ACCOUDOIR|AERATEURS|AIR      |ALARME DI|ANNONCEUR|ANNONCEUR|ARRET MOT|ARRET URG|ARTICULAT|AUTO-RADI|AVTT     |BAGUETTE |BARRE    |BOITE VIT|BOITIER G|BOITIER P|BOUTON/ P|BRUITEUR |CAOU.JOIN|CARBURANT|CHAUFFAGE|CLIGNOTAN|CLOCHE EC|COLMATAGE|COMBINE R|COMMANDE |COMMODO  |COMPTAGE |COMPTEUR/|DEGIVRAGE|DEMANDE A|DEMARRAGE|DEPART AU|DIRECTION|EAU      |ECHAPPEME|ECLAIRAGE|ECLAIRAGE|ESSUIE-GL|EXTINCTEU|FEU ANTIB|FEU CLIGN|FEU GABAR|FEU MARCH|FEU STOP |FREIN A P|FREIN DE |FREIN EXP|GIROUETTE|GIROUETTE|GLACE/BAI|HABILLAGE|HAUT-PARL|HUILE DIR|HUILE MOT|INFORMATI|ISSUE SEC|KLAXON/GO|LAVE-GLAC|LUBRIMATI|LUNETTE-A|MANOMETRE|MARCHE AR|MARTEAU B|MICRO    |MOTEUR   |PANNEAU  |PAPIER BU|PARE-BRIS|PARE-CHOC|PARE-JAMB|PARE-SOLE|PEDALE AC|PEDALE DE|PILE BTSA|POIGNEE  |PORTE    |PORTILLON|PUBLICITE|PUPITRE.I|PURGE POR|RADIO    |RALENTI  |RALENTISS|RAMPE UFR|RETROVISE|ROUE/PNEU|S A E I V|SECURITE |SELECTEUR|SELECTEUR|SELF-SERV|SIEGE MAC|SIEGE VOY|SIGNALEME|SOUFFLET |SUSPENSIO|SYSTEME D|TACHYGRAP|TEMOIN DI|TIROIR CA|TOIT     |TRAPPE GA|VENTILATI|VIDEO POR|VIDEO-SUR|VITESSE  |VOITURE  |VOLANT   |VOYAGEUR |VOYANT AL|VOYANT AL|VOYANT DE|VOYANT IT|
------||---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|
DIVERS|| 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  

In [27]:
bn.cpt("KM_GROUP")

(pyAgrum.Potential<double>@000002551F43AB30) 
      ||  KM_GROUP                                                           |
SYSTEM||C:0-50k  |C:100k-15|C:150k-20|C:200k-25|C:250k-30|C:300k+  |C:50k-100|
------||---------|---------|---------|---------|---------|---------|---------|
DIVERS|| 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  |
EQUIPE|| 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  |
EQUIPE|| 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  |
EQUIPE|| 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  |
EQUIPE|| 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  |
EQUIPE|| 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  |
EQUIPE|| 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  |
EQUIPE|| 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  |
EQUIPE|| 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  |
EQUIPE|| 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  |

Loi de probabilité de SYSTEM_N1 | SIG_OBS, KM_GROUP, SIG_ORGANE :

In [28]:
bn.cpt("SYSTEM_N1")

(pyAgrum.Potential<double>@000002551F43AD50) 
  SYSTEM_N1                                                                                        |
DIVERS   |EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|
---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|
 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  |

On remarque naturellement que les LPC ne contiennent pour l'instant que des zéros…

## Apprentissage des LPC

Pour apprendre les paramètres des LPC à partir des données, vous pouvez utiliser la classe gum.BNLearner et en particulier la méthode fitParameters:

In [29]:
joined_df_filtered['KM_GROUP'].cat.categories

Index(['C:0-50k', 'C:100k-150k', 'C:150k-200k', 'C:200k-250k', 'C:250k-300k',
       'C:300k+', 'C:50k-100k'],
      dtype='object')

In [30]:
learner = gum.BNLearner(joined_df_filtered[var_to_model])

In [31]:
learner

(pyAgrum.BNLearner<double>@000002554DF43680) Filename       : C:\Users\askel\AppData\Local\Temp\tmpzlzbprc0.csv
Size           : (506557,4)
Variables      : SYSTEM_N1[10], SIG_OBS[59], SIG_ORGANE[116], KM_GROUP[7]
Induced types  : True
Missing values : False
Algorithm      : MIIC
Score          : BDeu  (Not used for constraint-based algorithms)
Correction     : MDL  (Not used for score-based algorithms)
Prior          : -

In [32]:

learner.fitParameters(bn)

(pyAgrum.BNLearner<double>@000002554DF43680) Filename       : C:\Users\askel\AppData\Local\Temp\tmpzlzbprc0.csv
Size           : (506557,4)
Variables      : SYSTEM_N1[10], SIG_OBS[59], SIG_ORGANE[116], KM_GROUP[7]
Induced types  : True
Missing values : False
Algorithm      : MIIC
Score          : BDeu  (Not used for constraint-based algorithms)
Correction     : MDL  (Not used for score-based algorithms)
Prior          : -

Après apprentissage, nous obtenons les LPC suivantes. Loi de probabilité de SIG_OBS :

In [33]:
bn.cpt("SIG_OBS")

(pyAgrum.Potential<double>@000002551F43ABB0) 
      ||  SIG_OBS                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
SYSTEM||A-COUPS  |ABSENT   |ALLUME   |ARRACHE  |ASPIRER  |BALANCE  |BLESSE   |BLOQUE   |BROUTE   |BRUTAL   |BRUYANT  |CALE     |CASSE    |CLAQUE   |CLIGNOTE |CREVE    |DEBOITE  |DECLENCHE|DEGONFLE |DELOCALIS|DEREGLE  |DESSERRE |DUR      |ENFONCE  |FAUX CONT|FENDU    |FERME MAL|FONCTIONN|FUITE    |FUME/CHAU|INCOMPLET|INSUFFISA|INTERMITT|INTERRUPT|JEU      |LACERE   |MANQUE   |MAUVAIS  |N'EMET PA|N'ENCRE P|NE FONCTI|NE TIRE P|ODEUR    |OUVRE MAL|PATINE   |RAYE     |REFUSE TI|SALE     |SECOUSSE |TAGUE    |TANGUE   |TEMOIN OU|TIRE A DR|TIRE A GA|TORDU    |USE/VIDE |VIBRE    |VOYANT AL|VOYANT HS|
------||---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|
DIVERS|| 0.0213  | 0.0000  | 0.0000  | 0.0213  | 0.0000  | 0.0000  | 0.0000  | 0.0213  | 0.0000  | 0.0000  | 0.2979  | 0.0000  | 0.0213  | 0.0213  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0213  | 0.0000  | 0.0000  | 0.0000  | 0.2766  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.1277  | 0.0000  | 0.0213  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0000  | 0.0638  | 0.0000  | 0.0000  | 0.0000  | 0.0213  | 0.0638  | 0.0000  |
EQUIPE|| 0.0371  | 0.0029  | 0.0001  | 0.0067  | 0.0000  | 0.0344  | 0.0107  | 0.0750  | 0.0000  | 0.0000  | 0.1424  | 0.0006  | 0.0485  | 0.0604  | 0.0078  | 0.0142  | 0.0116  | 0.0001  | 0.1296  | 0.0000  | 0.0017  | 0.0059  | 0.0650  | 0.0119  | 0.0001  | 0.0001  | 0.0004  | 0.0822  | 0.0667  | 0.0054  | 0.0003  | 0.0011  | 0.0082  | 0.0014  | 0.0002  | 0.0038  | 0.0042  | 0.0000  | 0.0000  | 0.0000  | 0.0733  | 0.0007  | 0.0021  | 0.0001  | 0.0004  | 0.0088  | 0.0000  | 0.0000  | 0.0000  | 0.0023  | 0.0002  | 0.0006  | 0.0032  | 0.0017  | 0.0002  | 0.0085  | 0.0105  | 0.0451  | 0.0017  |
EQUIPE|| 0.0009  | 0.1313  | 0.0000  | 0.0001  | 0.0000  | 0.0003  | 0.0000  | 0.0382  | 0.0000  | 0.0000  | 0.0443  | 0.0001  | 0.0677  | 0.0006  | 0.0011  | 0.0001  | 0.0046  | 0.0000  | 0.0004  | 0.0000  | 0.0000  | 0.0005  | 0.0004  | 0.0010  | 0.0000  | 0.0000  | 0.0001  | 0.1216  | 0.0967  | 0.0155  | 0.0001  | 0.1347  | 0.0050  | 0.0196  | 0.0000  | 0.0003  | 0.0104  | 0.0000  | 0.0000  | 0.0000  | 0.2411  | 0.0003  | 0.0178  | 0.0000  | 0.0000  | 0.0010  | 0.0000  | 0.0009  | 0.0000  | 0.0000  | 0.0000  | 0.0001  | 0.0001  | 0.0001  | 0.0000  | 0.0000  | 0.0004  | 0.0411  | 0.0016  |
EQUIPE|| 0.0010  | 0.0279  | 0.0000  | 0.0247  | 0.0003  | 0.0013  | 0.0001  | 0.0596  | 0.0000  | 0.0002  | 0.0116  | 0.0001  | 0.3580  | 0.0103  | 0.0018  | 0.0001  | 0.0675  | 0.0000  | 0.0018  | 0.0000  | 0.0007  | 0.0229  | 0.0023  | 0.0505  | 0.0001  | 0.0007  | 0.0034  | 0.1064  | 0.0308  | 0.0008  | 0.0012  | 0.0094  | 0.0026  | 0.0139  | 0.0000  | 0

Loi de probabilité de SYSTEM_N1 | SIG_OBS :

In [34]:
bn.cpt("SYSTEM_N1")

(pyAgrum.Potential<double>@000002551F43AD50) 
  SYSTEM_N1                                                                                        |
DIVERS   |EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|EQUIPEMEN|
---------|---------|---------|---------|---------|---------|---------|---------|---------|---------|
 0.0001  | 0.0316  | 0.0270  | 0.2472  | 0.0232  | 0.0450  | 0.0027  | 0.5633  | 0.0405  | 0.0195  |

## Prédictions

À ce stade, vous pouvez reprendre la fonction predict que nous avons utilisée dans ce TD pour réaliser des prédictions d'une variable cible à partir de données. Chaque ligne des données fournies est utilisée comme évidence (conditionnement) pour calculer la loi de la variable cible sachant les données observées.

In [35]:
def predict(bn, data, var_target, show_progress=False):
    """
    This function is used to predict the posterior probability of a target variable from observations  
    using a bayesian network model. 

    Inputs:
    - =bn=: the predictive model given as a =pyAgrum.BayesNet= object
    - =data=: the data containing the observations used to predict the target variable 
    as a =pandas.DataFrame= object
    - =var_target=: the name of the target variable as a =str= object

    Returns:
    - a =DataFrame= containing the posterior probability distribution of the target 
    variable given each observation in =data=.
    """
    # Initialize the inference engine
    inf_bn = gum.LazyPropagation(bn)
    inf_bn.setTargets({var_target})
    nb_data = len(data)
    target_size = bn.variable(var_target).domainSize()
    target_dom = np.array([bn.variable(var_target).label(i)
                           for i in range(target_size)])
    data_records = data.to_dict("records")
    post_prob = np.zeros((nb_data, target_size))
    for i in range(nb_data):
        # Set the evidence
        inf_bn.setEvidence(data_records[i])
        # Run inference
        inf_bn.makeInference()
        # Compute posterior probability of target variable
        post_prob[i, :] = inf_bn.posterior(var_target).toarray()
        # Erase evidence
        inf_bn.eraseAllEvidence()
        if show_progress:
            sys.stdout.write("predict progress: {0:3.0%}\r".format(i/nb_data))

    post_prob_df = pd.DataFrame(post_prob,
                                index=data.index,
                                columns=bn.variable(var_target).labels())
    post_prob_df.columns.name = var_target
    return post_prob_df

In [36]:
pred_prob = predict(bn,
                    data=joined_df_filtered[["SIG_OBS", "SIG_ORGANE","KM_GROUP"]].iloc[-1000:], 
                    var_target="SYSTEM_N1",
                    show_progress=True)

pred_var = pred_prob.idxmax(axis=1).rename("pred")

On finit par évaluer le taux de prédictions du modèle par rapport aux données observées.

In [38]:
(joined_df_filtered["SYSTEM_N1"].iloc[-1000:] == pred_var).mean()

0.94

Et la matrice de confusion associée :

In [39]:
pd.crosstab(joined_df_filtered["SYSTEM_N1"].iloc[-1000:], pred_var)

pred,EQUIPEMENT CHASSIS,EQUIPEMENT CLIMATIQUE,EQUIPEMENT DE CARROSSERIE,EQUIPEMENT DE FREINAGE,EQUIPEMENT DE MOTORISATION,EQUIPEMENT ELECTRIQUE,EQUIPEMENT EMBARQUE,EQUIPEMENT PNEUMATIQUE
SYSTEM_N1,,,,,,,,
EQUIPEMENT CHASSIS,3,0,2,0,1,0,0,0
EQUIPEMENT CLIMATIQUE,0,74,3,0,0,2,0,0
EQUIPEMENT DE CARROSSERIE,1,0,232,0,3,6,1,1
EQUIPEMENT DE FREINAGE,0,0,0,5,1,0,0,0
EQUIPEMENT DE MOTORISATION,0,0,4,0,19,0,0,0
EQUIPEMENT DE TRANSMISSION,0,0,0,0,0,1,0,0
EQUIPEMENT ELECTRIQUE,0,1,22,1,7,493,0,0
EQUIPEMENT EMBARQUE,0,0,2,0,1,0,114,0
